# LangChain: Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest.

In [124]:
pip install --upgrade langchain

Note: you may need to restart the kernel to use updated packages.


In [125]:
import os

os.environ['OPENAI_API_KEY'] = "Enter Your API Key here..."

In [126]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [127]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file,encoding="utf-8")

In [128]:
from langchain.indexes import VectorstoreIndexCreator

In [129]:
pip install docarray

Note: you may need to restart the kernel to use updated packages.


In [130]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [131]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [132]:
response = index.query(query)

In [133]:
display(Markdown(response))



| Name | Description |
| --- | --- |
| Men's Tropical Plaid Short-Sleeve Shirt | UPF 50+ rated, 100% polyester, wrinkle-resistant, front and back cape venting, two front bellows pockets |
| Men's Plaid Tropic Shirt, Short-Sleeve | UPF 50+ rated, 52% polyester and 48% nylon, machine washable and dryable, front and back cape venting, two front bellows pockets |
| Men's TropicVibe Shirt, Short-Sleeve | UPF 50+ rated, 71% Nylon, 29% Polyester, 100% Polyester knit mesh, machine wash and dry, front and back cape venting, two front bellows pockets |
| Sun Shield Shirt by | UPF 50+ rated, 78% nylon, 22% Lycra Xtra Life fiber, handwash, line dry, wicks moisture, fits comfortably over swimsuit, abrasion resistant |

All four shirts provide UPF 50+ sun protection, blocking 98% of the sun's harmful rays. The Men's Tropical Plaid Short-Sleeve Shirt is made of 100% polyester and is wrinkle-resistant

In [134]:
loader = CSVLoader(file_path=file,encoding="utf-8")

In [135]:
docs = loader.load()

In [136]:
docs[0]

Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0})

In [137]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [138]:
embed = embeddings.embed_query("Hi my name is Harrison")

In [139]:
print(len(embed))

1536


In [140]:
print(embed[:5])

[-0.021930990740656853, 0.006712669972330332, -0.018181458115577698, -0.039156194776296616, -0.014079621061682701]


In [141]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [142]:
# query = "Please suggest a shirt with sunblocking"
query = "What is the best name to describe a company that makes Queen Size Sheet Set?"

In [143]:
docs = db.similarity_search(query)

In [144]:
len(docs)

4

In [145]:
docs[0]

Document(page_content=': 904\nname: Wrinkle-Resistant Sheet Collection, Floral\ndescription: Our luxurious wrinkle-free floral sheet set is made from soft, 300-thread-count sateen fabric, ensuring a smooth finish without the need for ironing. The set includes one flat sheet, one fitted sheet and two pillowcases. Twin has one case. Twin set sizes include a flat sheet measuring 70" x 105", a fitted sheet measuring 39" x 76" with a pocket depth of 15", and pillowcases measuring 21" x 31". Full set sizes include a flat sheet measuring 84" x 105", a fitted sheet measuring 54" x 76" with a pocket depth of 15", and pillowcases measuring 21" x 31". Queen set sizes include a flat sheet measuring 92" x 110", a fitted sheet measuring 60" x 80" with a pocket depth of 15", and pillowcases measuring 21" x 31". King set sizes include a flat sheet measuring 108" x 110", a fitted sheet measuring 78" x 80" with a pocket depth of 15", and pillowcases measuring 21" x 41". Standard pillowcases (2) dimensio

In [146]:
retriever = db.as_retriever()

In [147]:
llm = ChatOpenAI(temperature = 0.0)


In [148]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])


In [149]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 


In [150]:
display(Markdown(response))

I'm sorry, but there are no shirts mentioned in the previous product descriptions.

In [151]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [152]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [153]:
response = qa_stuff.run(query)



> Entering new  chain...

> Finished chain.


In [154]:
display(Markdown(response))

| Shirt Number | Name | Description |
| --- | --- | --- |
| 618 | Men's Tropical Plaid Short-Sleeve Shirt | This shirt is made of 100% polyester and is wrinkle-resistant. It has front and back cape venting that lets in cool breezes and two front bellows pockets. It is rated UPF 50+ for superior protection from the sun's UV rays. |
| 374 | Men's Plaid Tropic Shirt, Short-Sleeve | This shirt is made with 52% polyester and 48% nylon. It is machine washable and dryable. It has front and back cape venting, two front bellows pockets, and is rated to UPF 50+. |
| 535 | Men's TropicVibe Shirt, Short-Sleeve | This shirt is made of 71% Nylon and 29% Polyester. It has front and back cape venting that lets in cool breezes and two front bellows pockets. It is rated UPF 50+ for superior protection from the sun's UV rays. |
| 255 | Sun Shield Shirt | This shirt is made of 78% nylon and 22% Lycra Xtra Life fiber. It is handwashable and line dry. It is rated UPF 50+ for superior protection from the sun's UV rays. It is abrasion-resistant and wicks moisture for quick-drying comfort. |

The Men's Tropical Plaid Short-Sleeve Shirt is made of 100% polyester and is wrinkle-resistant. It has front and back cape venting that lets in cool breezes and two front bellows pockets. It is rated UPF 50+ for superior protection from the sun's UV rays.

The Men's Plaid Tropic Shirt, Short-Sleeve is made with 52% polyester and 48% nylon. It has front and back cape venting, two front bellows pockets, and is rated to UPF 50+.

The Men's TropicVibe Shirt, Short-Sleeve is made of 71% Nylon and 29% Polyester. It has front and back cape venting that lets in cool breezes and two front bellows pockets. It is rated UPF 50+ for superior protection from the sun's UV rays.

The Sun Shield Shirt is made of 78% nylon and 22% Lycra Xtra Life fiber. It is abrasion-resistant and wicks moisture for quick-drying comfort. It is rated UPF 50+ for superior protection from the sun's UV rays. It is handwashable and line dry.

In [155]:
response = index.query(query, llm=llm)

In [156]:
# index = VectorstoreIndexCreator(
#     vectorstore_cls=DocArrayInMemorySearch,
#     embedding=embeddings,
# ).from_loaders([loader])